In [1]:
import os
from os import path
import sys

import json
import re
import torch

import uuid

from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams
from qdrant_client.http import models
from qdrant_client.http.models import CollectionStatus
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams

import transformers
from sentence_transformers import SentenceTransformer

from tqdm import tqdm

import pickle

if torch.cuda.is_available():
    for i in range(torch.cuda.device_count()):
        print(f"Device {i}: {torch.cuda.get_device_name(i)}")
else:
    print("CUDA is not available.")

device = "cuda" if torch.cuda.is_available() else "cpu"

print(f"using {device} as device")

Device 0: NVIDIA RTX A5000
using cuda as device


# Load creditreform metadata

In [2]:
BASEDIR = "/local/innecs-local/website-data"

f = open(path.join(BASEDIR, "extracted_sachsen_anhalt.json"))
data = json.loads(f.read())
f.close()
crefodict = {item['crefonummer']: item for item in data}

# Load website data and embedd

In [3]:
# qdrant = QdrantClient(url="http://qdrant:6333")
qdrant = QdrantClient(host="172.18.0.4", port=6333)

qdrant.recreate_collection(
    collection_name="ies_website_embeddings_chunks",
    vectors_config=VectorParams(size=384, distance=Distance.COSINE),
)

model = SentenceTransformer("all-MiniLM-L6-v2")

/tmp/ipykernel_468/3053773973.py:4: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  qdrant.recreate_collection(
/usr/local/lib/python3.8/dist-packages/transformers/tokenization_utils_base.py:1614: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [4]:
text = "Depending on the requirements of the application, Qdrant can use one of the data storage options."
encoded = model.tokenizer(text, return_offsets_mapping=True)
print(encoded)

print(text[13:88])
# tokens = model.tokenizer.encode(text)[1:-1]
# print("offset mapping:", tokens["offset_mapping"])
# print(tokens)
# print(model.tokenizer.decode(tokens))
# print([[101] + tokens + [102]])
min(223,43)

{'input_ids': [101, 5834, 2006, 1996, 5918, 1997, 1996, 4646, 1010, 1053, 24914, 2102, 2064, 2224, 2028, 1997, 1996, 2951, 5527, 7047, 1012, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'offset_mapping': [(0, 0), (0, 9), (10, 12), (13, 16), (17, 29), (30, 32), (33, 36), (37, 48), (48, 49), (50, 51), (51, 55), (55, 56), (57, 60), (61, 64), (65, 68), (69, 71), (72, 75), (76, 80), (81, 88), (89, 96), (96, 97), (0, 0)]}
the requirements of the application, Qdrant can use one of the data storage


43

In [5]:
error_counter = 0

def create_sliding_windows(text, model, max_length=-1, overlap=-1):
    CLS = model.tokenizer.encode("[CLS]")
    SEP = model.tokenizer.encode("[SEP]")
    
    if max_length < 0:
        max_length = model.max_seq_length - 2 # -2 to account for CLS and SEP
    if overlap < 0:
        overlap = int(model.max_seq_length / 2)

    encoded = model.tokenizer(text, return_offsets_mapping=True)
    tokens = encoded["input_ids"][1:-1]
    offsets = encoded["offset_mapping"][1:-1]
        
    sliding_windows = []
    
    for i in range(1, len(tokens), max_length - overlap):
        idx0 = offsets[i][0]
        idx1 = offsets[min(i + max_length, len(tokens)-1)][1]
        window = text[idx0:idx1]
        if len(window) > 0: # only if the window contains
            sliding_windows.append(window)

    return sliding_windows


def handle_websitefolder(websitefolder, crefodata):
    global error_counter

    no_prev_entries = 0
    
    for file in os.listdir(websitefolder):
        filepath = path.join(websitefolder, file)
        if path.isfile(filepath) and filepath.endswith(".txt"):
            with open(filepath, "r") as f:
                sliding_windows = create_sliding_windows(f.read(), model)
                if len(sliding_windows) > 0:                
                    embeddings = model.encode(sliding_windows)                                        
                            
                payloads = [{
                        "filepath": filepath, 
                        "filename": file, 
                        "crefoid": crefodata["crefonummer"], 
                        "name": crefodata["name"], 
                        "idx": i, 
                        "text": w, 
                        "crefodata": crefodata
                    } for i, w in enumerate(sliding_windows)]
                
                ids = [(no_prev_entries + i + 1) for i in range(len(payloads))]
                no_prev_entries += len(payloads)

                if payloads and len(payloads) > 0:                    
                    try:
                        qdrant.upsert(
                            collection_name="ies_website_embeddings_chunks",
                            points=models.Batch(
                                ids = [str(uuid.uuid4()) for i in range(len(payloads))],
                                vectors=embeddings,
                                payloads=payloads
                            )
                        )
                    except:
                        error_counter += 1
                        with open(f"logs/error-payloads_{error_counter}.pkl", "wb") as f:
                            pickle.dump(payloads, f)    
                else:
                    print("empty payload on file:", filepath)
                    with open("logs/empty_payloads_files.log", "a") as f:
                        f.write(filepath)
                        f.write("\n")

with open("logs/progress.log", "w") as progress_log:        
    for folder in tqdm(os.listdir(BASEDIR), file=progress_log):
        websitefolder = path.join(BASEDIR, folder)
        if path.isdir(websitefolder):            
            crefoid = folder[0: folder.find("_")]                        
            crefodata = crefodict[crefoid]
            handle_websitefolder(websitefolder, crefodata)        

Token indices sequence length is longer than the specified maximum sequence length for this model (1581 > 256). Running this sequence through the model will result in indexing errors


empty payload on file: /local/innecs-local/website-data/3370068301_zimmerei-kupfer-de/disclaimer-index-html.txt
empty payload on file: /local/innecs-local/website-data/3170167567_schoenebeck-de/de-instagram-html.txt
empty payload on file: /local/innecs-local/website-data/3130030555_bs-line-com/wp-admin-admin-ajax-php-p-27.txt
empty payload on file: /local/innecs-local/website-data/3130030555_bs-line-com/wp-admin-admin-ajax-php-p-169.txt
empty payload on file: /local/innecs-local/website-data/3130030555_bs-line-com/wp-admin-admin-ajax-php-p-111.txt
empty payload on file: /local/innecs-local/website-data/3130030555_bs-line-com/wp-admin-admin-ajax-php-p-112.txt
empty payload on file: /local/innecs-local/website-data/3130030555_bs-line-com/wp-admin-admin-ajax-php-p-233.txt
empty payload on file: /local/innecs-local/website-data/3130030555_bs-line-com/wp-admin-admin-ajax-php-p-167.txt
empty payload on file: /local/innecs-local/website-data/3130030555_bs-line-com/wp-admin-admin-ajax-php-p-48